# Exploration 14
## recommendation system for Movies
---
평가문항|상세기준
------|------
CSR matrix가 정상적으로 만들어졌다.|사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.
MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.|사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.
비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.|MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. MovieLens 1M Dataset 사용을 권장한다.
* 별점 데이터는 대표적인 explicit 데이터다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있다.
* 별점을 시청횟수로 해석해서 생각하자.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠다.

### 1) 데이터 준비와 전처리
* Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# timestamp 컬럼은 우리의 관심사에서 제외시키도록 하겠다.
using_cols = ['user_id', 'movie_id', 'ratings']
ratings = ratings[using_cols]
ratings.head()

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### pd.merge를 활용하여 ratings 와 movies 두 데이터 프레임을 합쳐주었다. 
* 공통의 열 'movie_id'를 기준으로
* 새로운 데이터프레임 : rt_mv

In [7]:
rt_mv = pd.merge(ratings, movies, on = 'movie_id', how = 'left')
rt_mv.head(30)

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
5,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1,1287,5,Ben-Hur (1959),Action|Adventure|Drama
7,1,2804,5,"Christmas Story, A (1983)",Comedy|Drama
8,1,594,4,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical
9,1,919,4,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical


In [8]:
# movie_id 는 title로 대체가능하기 때문에 제거하도록 하겠다.
# genre는 여기서 다룰 항목이 아니므로 제거하도록 하겠다.
using_cols = ['user_id', 'title', 'counts']
rt_mv = rt_mv[using_cols]
rt_mv.head(30)

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,1,James and the Giant Peach (1996),3
2,1,My Fair Lady (1964),3
3,1,Erin Brockovich (2000),4
4,1,"Bug's Life, A (1998)",5
5,1,"Princess Bride, The (1987)",3
6,1,Ben-Hur (1959),5
7,1,"Christmas Story, A (1983)",5
8,1,Snow White and the Seven Dwarfs (1937),4
9,1,"Wizard of Oz, The (1939)",4


In [9]:
# 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
rt_mv['title'] = rt_mv['title'].str.lower()
rt_mv.head(10)

,user_id,title,counts
0,1,one flew over the cuckoo's nest (1975),5
1,1,james and the giant peach (1996),3
2,1,my fair lady (1964),3
3,1,erin brockovich (2000),4
4,1,"bug's life, a (1998)",5
5,1,"princess bride, the (1987)",3
6,1,ben-hur (1959),5
7,1,"christmas story, a (1983)",5
8,1,snow white and the seven dwarfs (1937),4
9,1,"wizard of oz, the (1939)",4


### 2) 분석해 봅시다.
* rt_mv에 있는 유니크한 영화 개수
* rt_mv에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [10]:
# 영화 개수
rt_mv['title'].nunique()

3628

In [11]:
# 사용자 수
rt_mv['user_id'].nunique()

6039

In [12]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = rt_mv.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(60)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
* 단, 영화 title은 꼭 데이터셋에 있는 것과 동일하게 맞춰주자!

In [13]:
my_favorite = ['toy story (1995)', 'back to the future (1985)', '2001: a space odyssey (1968)', 'when harry met sally... (1989)', 'shawshank redemption, the (1994)']

# 'ahyun' 이라는 user_id가 위 title의 영화를 4회씩 들었다고 가정하자.
my_movielist = pd.DataFrame({'user_id':['ahyun']*5, 'title':my_favorite, 'counts':[4]*5})

# user_id에 'ahyun'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 준다.
if not rt_mv.isin({'user_id':['ahyun']})['user_id'].any():
    rt_mv = rt_mv.append(my_movielist)
    
rt_mv.tail(10)

,user_id,title,counts
836473,6040,platoon (1986),3
836474,6040,"crying game, the (1992)",5
836475,6040,welcome to the dollhouse (1995),5
836476,6040,sophie's choice (1982),4
836477,6040,e.t. the extra-terrestrial (1982),4
0,ahyun,toy story (1995),4
1,ahyun,back to the future (1985),4
2,ahyun,2001: a space odyssey (1968),4
3,ahyun,when harry met sally... (1989),4
4,ahyun,"shawshank redemption, the (1994)",4


In [14]:
# 고유한 유전, 영화를 찾아내는 코드
user_unique = rt_mv['user_id'].unique()
title_unique = rt_mv['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다.
print(user_to_idx['ahyun']) # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 한다.
print(title_to_idx['toy story (1995)'])

6039
40


In [16]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해보자
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NsN이 될 테니 dropna()로 제거하자.
temp_user_data = rt_mv['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(rt_mv):
    print('user_id column indexing OK!!!')
    rt_mv['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!!')
    
# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해주자.
temp_title_data = rt_mv['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(rt_mv):
    print('title column indexing OK!!!')
    rt_mv['title'] = temp_title_data
else:
    print('title column indexing Fail!!!')

rt_mv

user_id column indexing OK!!!
title column indexing OK!!!


,user_id,title,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,40,4
1,6039,22,4
2,6039,602,4
3,6039,488,4


### 4) CSR matrix를 직접 만들어 봅시다.

In [17]:
from scipy.sparse import csr_matrix

num_user = rt_mv['user_id'].nunique()
num_title = rt_mv['title'].nunique()

csr_data = csr_matrix((rt_mv.counts, (rt_mv.user_id, rt_mv.title)), shape=(num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [19]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [20]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
ahyun, toy_story = user_to_idx['ahyun'], title_to_idx['toy story (1995)']
ahyun_vector, toy_story_vector = als_model.user_factors[ahyun], als_model.item_factors[toy_story]

In [23]:
ahyun_vector

array([-0.5519364 , -0.6155268 ,  0.02045531, -0.68077177,  0.37783578,
        0.02065604, -0.0983932 ,  0.04068447,  0.03812863,  0.16749686,
        0.36838   ,  0.8066984 , -0.2862315 , -0.04365909,  0.13470523,
       -1.145414  , -0.6235364 , -0.44660988,  0.24075869,  0.29848653,
        0.0805095 ,  0.7144845 , -0.7532338 ,  0.63624805,  0.3882481 ,
       -0.03956684, -1.0516667 , -0.7213596 ,  0.25977516, -0.09056551,
       -0.8322688 , -0.25020888,  0.78021556,  0.24264815, -0.0986532 ,
        0.40459144, -0.46540254,  0.31213254, -0.18763639,  0.75345135,
       -0.3800125 , -0.2361536 , -0.23557034,  0.08167735,  0.32864586,
       -0.58978695,  0.5056983 ,  0.54759836, -0.02963037, -0.21973158,
        0.00522105,  0.74606305,  0.93425167,  0.11985645,  0.05898516,
        0.30750155, -0.23375562,  0.35358563, -0.15861581, -0.09386273,
       -0.16113412, -0.69836694,  0.24591626, -0.28632134,  0.49735546,
        0.9395974 , -0.1254502 , -0.07149595, -0.16942452,  0.02

In [24]:
toy_story_vector

array([ 0.00142636,  0.03238982,  0.01176937,  0.01340075,  0.0014439 ,
       -0.00785363,  0.00031495,  0.01315196,  0.0067949 , -0.00090274,
       -0.01325936,  0.02525725, -0.01002829,  0.0073875 ,  0.00588219,
       -0.01890689, -0.02039837, -0.01114533,  0.00087805, -0.0004784 ,
        0.01621058,  0.00962516,  0.00025802,  0.01633367,  0.02584401,
        0.01367853,  0.00789963,  0.01058006, -0.00032572,  0.01996089,
       -0.05497993,  0.02434116,  0.02619427,  0.00801575,  0.02329165,
       -0.00876183, -0.01237181,  0.01934108,  0.00893245,  0.01599179,
       -0.00992298,  0.01559236,  0.00368815,  0.00215699,  0.01389837,
       -0.00216383,  0.03487716,  0.01240564,  0.00193728,  0.014104  ,
        0.01143018,  0.00688572,  0.00284092, -0.0136064 , -0.00135693,
       -0.00845369, -0.01749739,  0.02570187, -0.00851188,  0.00408189,
        0.0256232 , -0.01806619,  0.02085276,  0.02185235,  0.01357766,
        0.04526904,  0.0100461 ,  0.01152753, -0.01618409,  0.02

In [25]:
# ahyun과 toy_story를 내적하는 코드
np.dot(ahyun_vector, toy_story_vector)

0.44710055

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [26]:
back_to_the_future = title_to_idx['back to the future (1985)']
back_to_the_future_vector = als_model.item_factors[back_to_the_future]
np.dot(ahyun_vector, back_to_the_future_vector)

0.61254907

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [27]:
favorite_title = '2001: a space odyssey (1968)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=10)
similar_title

[(602, 1.0),
 (91, 0.674102),
 (550, 0.6441134),
 (904, 0.5654598),
 (876, 0.47261328),
 (193, 0.4724308),
 (569, 0.43135893),
 (1097, 0.43100682),
 (1300, 0.41900507),
 (74, 0.40603268)]

In [28]:
# title_to_idx를 뒤집어, index로부터 title 이름을 얻는 dict를 생성하자.
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['2001: a space odyssey (1968)',
 'close encounters of the third kind (1977)',
 'dr. strangelove or: how i learned to stop worrying and love the bomb (1963)',
 'clockwork orange, a (1971)',
 '2010 (1984)',
 'alien (1979)',
 'rear window (1954)',
 'brazil (1985)',
 'apocalypse now (1979)',
 'day the earth stood still, the (1951)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [29]:
user = user_to_idx['ahyun']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
title_recommended

[(50, 0.40051746),
 (121, 0.35175788),
 (23, 0.3459038),
 (110, 0.3011735),
 (160, 0.29439044),
 (248, 0.27694607),
 (13, 0.27116916),
 (322, 0.27028286),
 (91, 0.26899847),
 (680, 0.21503372),
 (4, 0.21464503),
 (243, 0.21324158),
 (0, 0.20815441),
 (332, 0.20576438),
 (904, 0.20324594)]

In [30]:
[idx_to_title[i[0]] for i in title_recommended]

['toy story 2 (1999)',
 'silence of the lambs, the (1991)',
 "schindler's list (1993)",
 'groundhog day (1993)',
 'forrest gump (1994)',
 'good will hunting (1997)',
 "ferris bueller's day off (1986)",
 'babe (1995)',
 'close encounters of the third kind (1977)',
 'blade runner (1982)',
 "bug's life, a (1998)",
 'ghostbusters (1984)',
 "one flew over the cuckoo's nest (1975)",
 'moonstruck (1987)',
 'clockwork orange, a (1971)']

In [31]:
schindlers_list = title_to_idx["schindler's list (1993)"]
explain = als_model.explain(user, csr_data, itemid=schindlers_list)

In [32]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('shawshank redemption, the (1994)', 0.2845595074406771),
 ('when harry met sally... (1989)', 0.037906350115240385),
 ('toy story (1995)', 0.017487983516753694),
 ('back to the future (1985)', 0.0012609385327574528),
 ('2001: a space odyssey (1968)', 0.0009253725308718697)]

#### 정리
* schindler's list를 추천받았다.
* AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인해 볼 수 있다.
* explain 메서드는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환한다. 어떤 영화들이 이 추천에 얼마나 기여하고 있는 것일까?
* 'shawshank redemption, the (1994)'와 'when harry met sally... (1989)'가 가장 크게 기여했네요.(솔직히 'when harry met sally... (1989)'는 기여도가 0.032 밖에 되지 않는데 기여를 했다고 할 수 있는 수치인가...?)